### Calculator / Eval

In [ ]:
def evaluate_expression(expr: str) -> float:
    """
    Evaluates a string expression containing +, -, *, /, and parentheses.
    Example: "-5 + 12/(23/2 +(11/9-52-4)) * 7 - 3"
    """
    # --- Tokenize the expression ---
    tokens: list[str] = []
    i = 0
    while i < len(expr):
        if expr[i].isspace():
            # Skip
            i += 1
            continue
        if expr[i] in "+-*/()":
            # Operator is a token alone
            tokens.append(expr[i])
            i += 1
        elif expr[i].isdigit() or expr[i] == '.':
            # Parse out the number
            num_start = i
            while i < len(expr) and (expr[i].isdigit() or expr[i] == '.'):
                i += 1
            tokens.append(expr[num_start:i])
        else:
            raise ValueError(f"Unexpected character: {expr[i]}")
    # We'll parse from this token list
    index = 0

    def peek() -> str | None:
        return tokens[index] if index < len(tokens) else None

    def get_next() -> str | None:
        nonlocal index
        token = tokens[index] if index < len(tokens) else None
        index += 1
        return token

    # --- Grammar:
    # expression = term { ('+' | '-') term }
    # term       = factor { ('*' | '/') factor }
    # factor     = number | '(' expression ')' | unary minus
    # -----------------------------------------------------

    def parse_expression():
        value = parse_term()
        while True:
            current = peek()
            if current == '+' or current == '-':
                op = get_next()
                right = parse_term()
                value = value + right if op == '+' else value - right
            else:
                break
        return value

    def parse_term():
        value = parse_factor()
        while True:
            current = peek()
            if current == '*' or current == '/':
                op = get_next()
                right = parse_factor()
                value = value * right if op == '*' else value / right
            else:
                break
        return value

    def parse_factor():
        current = peek()
        # Handle unary minus
        if current == '-':
            get_next()  # consume '-'
            return -parse_factor()
        # Parentheses
        if current == '(':
            get_next()  # consume '('
            val = parse_expression()
            if peek() != ')':
                raise ValueError("Missing closing parenthesis")
            get_next()  # consume ')'
            return val
        # Numeric
        token = get_next()
        try:
            return float(token)
        except ValueError:
            raise ValueError(f"Expected number, got '{token}'")

    # Parse and return result
    result = parse_expression()
    return result


In [32]:
expressions =[
    "-(5 + 12/(23/2 +(11/9-52-4)) * 7) - 3",
    "-5 + 12.6/(23/2 +(11/9-52-4)) * 7 - 3",
    "(-((5+1)/5*2)-2)"

]
for s in expressions:
    print(eval(s), evaluate_expression(s))

-6.059050064184852 -6.059050064184852
-10.037997432605906 -10.037997432605906
-4.4 -4.4
